In [1]:
from QuantLib import *

In [2]:
today = Date(24, 12, 2016)
Settings.instance().evaluationDate = today

In [3]:
u = SimpleQuote(100.0)
r = SimpleQuote(0.01)
sigma = SimpleQuote(0.20)

risk_free_curve = FlatForward(today, QuoteHandle(r), Actual360())
volatility = BlackConstantVol(today, TARGET(), QuoteHandle(sigma), Actual360())

In [10]:
process_1 = BlackScholesProcess(QuoteHandle(u), YieldTermStructureHandle(risk_free_curve), BlackVolTermStructureHandle(volatility))

print (process_1.dividendYield().zeroRate(1.0, Continuous).rate())

0.0


In [11]:
process_2 = BlackProcess(QuoteHandle(u), YieldTermStructureHandle(risk_free_curve), BlackVolTermStructureHandle(volatility))

print (process_2.riskFreeRate().zeroRate(1.0, Continuous).rate())
print (process_2.dividendYield().zeroRate(1.0, Continuous).rate())

0.009999999999999893
0.009999999999999893


In [12]:
option_1 = EuropeanOption(PlainVanillaPayoff(Option.Call, 100.0), EuropeanExercise(today + 100))
option_1.setPricingEngine(AnalyticEuropeanEngine(process_1))

print(option_1.NPV())

4.337597216336533


In [13]:
option_2 = EuropeanOption(PlainVanillaPayoff(Option.Call, 100.0), EuropeanExercise(today + 100))
option_2.setPricingEngine(AnalyticEuropeanEngine(process_2))

print(option_2.NPV())

4.191615257389808


In [14]:
def greek(option, quote, dx):
    x0 = quote.value()
    quote.setValue(x0 + dx)
    P_u = option.NPV()
    quote.setValue(x0 - dx)
    P_d = option.NPV()
    quote.setValue(x0)
    return (P_u - P_d)/ (2*dx)

In [17]:
print(option_1.delta())
print(greek(option_1, u, 0.01))

0.5315063340142601
0.531506323010289


In [18]:
print(option_2.delta())
print(greek(option_2, u, 0.01))

0.5195711146255227
0.5195711052036867


In [19]:
print(option_1.vega())
print(greek(option_1, sigma, 0.01))

20.96050033373808
20.96045787132743


In [20]:
print(option_2.vega())
print(greek(option_2, sigma, 0.01))

20.938677847075486
20.938653679838048


In [21]:
print(option_1.rho())
print(greek(option_1, r, 0.01))

13.559176718080407
13.55895831082643


In [22]:
print(option_2.rho())
print(greek(option_2, r, 0.01))

13.268193390322908
-1.1643390688454769


In [23]:
print(option_2.rho() + option_2.dividendRho())
print(greek(option_2, r, 0.01))

-1.1643375714971693
-1.1643390688454769


Black_scholes process, dividends q(t) = 0  
Black proceess, q(t) = r(t), which makes cost of carry b equal 0

In [24]:
# Black-Scholes-Merton Process

process_3 = BlackScholesMertonProcess(QuoteHandle(u),
                                      YieldTermStructureHandle(risk_free_curve),
                                      YieldTermStructureHandle(risk_free_curve),
                                      BlackVolTermStructureHandle(volatility))

option_3 = EuropeanOption(PlainVanillaPayoff(Option.Call, 100.0), EuropeanExercise(today + 100))
option_3.setPricingEngine(AnalyticEuropeanEngine(process_3))

In [25]:
print(option_3.rho())
print(greek(option_3, r, 0.001))
print(option_3.rho() + option_3.dividendRho())

13.268193390322908
-1.1643375864700545
-1.1643375714971693
